<a href="https://colab.research.google.com/github/emmyxth/2023-Utah/blob/main/end2end_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sympy==1.11
!pip install numpy
!pip install scipy
!pip install tabulate


  Using cached torch-2.5.1-cp310-cp310-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.21.5-py3-none

In [ ]:
!pip install torchvision==0.19

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 872.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import torch;
torch.__version__

'2.5.1+cu121'

In [ ]:
!pip install depyf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.1 MB/s eta 0:00:00


In [ ]:
import depyf
depyf.install()

ModuleNotFoundError: No module named 'depyf'

In [7]:
import torch
def fn(x):
   a = torch.cos(x)
   b = torch.sin(a)
   return b
new_fn = torch.compile(fn, backend="inductor")
input_tensor = torch.randn(10000)
a = new_fn(input_tensor)

In [25]:
import torch.nn as nn
from torch.fx.passes.graph_drawer import FxGraphDrawer

def dynamo_backend(gm, sample_inputs):
    code = gm.print_readable()
    gm.graph.print_tabular()
    with open("transformer_graph.svg", "wb") as file:
        file.write(FxGraphDrawer(gm,'f').get_dot_graph().create_svg())
    return gm.forward

In [26]:
from transformers import BertTokenizer, BertModel
# Copy pasted from here https://huggingface.co/bert-base-uncased

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
model = torch.compile(model, backend=dynamo_backend)
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

class GraphModule(torch.nn.Module):
    def forward(self, L_input_ids_: "i64[1, 12]", L_token_type_ids_: "i64[1, 12]", L_self_modules_embeddings_buffers_position_ids_: "i64[1, 512]", L_self_modules_embeddings_modules_word_embeddings_parameters_weight_: "f32[30522, 768]", L_self_modules_embeddings_modules_token_type_embeddings_parameters_weight_: "f32[2, 768]", L_self_modules_embeddings_modules_position_embeddings_parameters_weight_: "f32[512, 768]", L_self_modules_embeddings_modules_LayerNorm_parameters_weight_: "f32[768]", L_self_modules_embeddings_modules_LayerNorm_parameters_bias_: "f32[768]", L_attention_mask_: "i64[1, 12]", L_self_modules_encoder_modules_layer_modules_0_modules_attention_modules_self_modules_query_parameters_weight_: "f32[768, 768]", L_self_modules_encoder_modules_layer_modules_0_modules_attention_modules_self_modules_query_parameters_bias_: "f32[768]", L_self_modules_encoder_modules_layer_modules_0_modules_attention_modules_self_modules_key_parameters_weight_: "f

In [22]:
model

OptimizedModule(
  (_orig_mod): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
with depyf.prepare_debug("depyf_debug_dir"):
  output = model(**encoded_input)

:0: UserWarning: /usr/local/lib/python3.10/dist-packages/depyf/explain/enable_debugging.py:163: You are trying to debug `torch.compile`. Please make sure the code runs multiple times to cover all the possible branches.


In [ ]:
!zip -r /content/depyf_debug.zip /content/depyf_debug_dir/

  adding: content/depyf_debug_dir/ (stored 0%)
  adding: content/depyf_debug_dir/__compiled_fn_9.Forward_graph.0.py (deflated 68%)
  adding: content/depyf_debug_dir/full_code_for_forward_2.py (deflated 68%)
  adding: content/depyf_debug_dir/full_code_for_forward_1.py (deflated 92%)
  adding: content/depyf_debug_dir/__transformed_code_2_for_forward.py (deflated 33%)
  adding: content/depyf_debug_dir/full_code_for_forward_0.py (deflated 74%)
  adding: content/depyf_debug_dir/__compiled_fn_7.AFTER_POST_GRAD.0.py (deflated 76%)
  adding: content/depyf_debug_dir/__compiled_fn_9.Captured_Graph.0.py (deflated 66%)
  adding: content/depyf_debug_dir/__compiled_fn_14.Joint_graph.0.py (deflated 76%)
  adding: content/depyf_debug_dir/__compiled_fn_9.AFTER_POST_GRAD.0.py (deflated 66%)
  adding: content/depyf_debug_dir/__transformed_code_0_for_warn_if_padding_and_no_attention_mask.py (deflated 31%)
  adding: content/depyf_debug_dir/__compiled_fn_7.Captured_Graph.0.py (deflated 80%)
  adding: conten